<a href="https://colab.research.google.com/github/Manishkumarsingh41/LuminaPath-Ai-Retinal-Disease-Detection/blob/main/Traning_Retinal_Disease_Detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()  # select kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"manish5323","key":"99d5ed95844b2bef07edccd40e3eec19"}'}

In [ ]:
!pip install kaggle


In [ ]:
import os
os.makedirs("/root/.kaggle", exist_ok=True)
os.rename("kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 0o600)


In [ ]:
!kaggle --version


Kaggle API 1.7.4.5


In [ ]:
!kaggle datasets download -d obulisainaren/retinal-oct-c8


Dataset URL: https://www.kaggle.com/datasets/obulisainaren/retinal-oct-c8
License(s): CC-BY-NC-SA-4.0
100% 1.41G/1.42G [00:13<00:00, 197MB/s]
100% 1.42G/1.42G [00:13<00:00, 110MB/s]


In [ ]:
!unzip -q retinal-oct-c8.zip -d /content/OCT_C8

In [ ]:
import os
import shutil
import random

# Base dataset path
base_dir = '/content/OCT_C8'

In [ ]:
# Create train/val folders
train_dir = '/content/OCT_C8/train'
val_dir = '/content/OCT_C8/val'
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)


In [ ]:
# List of classes
classes = [cls for cls in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, cls))]


In [ ]:

# Create class folders inside train and val
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

In [ ]:
# Split images 80:20
for cls in classes:
    cls_path = os.path.join(base_dir, cls)
    images = [img for img in os.listdir(cls_path) if img.endswith(('.png','.jpg','.jpeg'))]
    random.shuffle(images)
    split_idx = int(0.8 * len(images))
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    for img in train_images:
        shutil.copy(os.path.join(cls_path, img), os.path.join(train_dir, cls, img))
    for img in val_images:
        shutil.copy(os.path.join(cls_path, img), os.path.join(val_dir, cls, img))

print("Dataset successfully split into train (80%) and validation (20%)!")

Dataset successfully split into train (80%) and validation (20%)!


In [ ]:
import os
import shutil
import random

base_dir = '/content/RetinalOCT_Dataset'

# Create train and val directories
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# List class folders
classes = [cls for cls in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, cls)) and cls not in ['train','val']]

# Split images 80:20 for each class
for cls in classes:
    cls_path = os.path.join(base_dir, cls)
    images = [img for img in os.listdir(cls_path) if img.endswith(('.jpg','.jpeg','.png'))]
    random.shuffle(images)
    split_idx = int(0.8 * len(images))

    # Create class folders in train/val
    train_cls_dir = os.path.join(train_dir, cls)
    val_cls_dir = os.path.join(val_dir, cls)
    os.makedirs(train_cls_dir, exist_ok=True)
    os.makedirs(val_cls_dir, exist_ok=True)

    # Move images
    for img in images[:split_idx]:
        shutil.move(os.path.join(cls_path, img), os.path.join(train_cls_dir, img))
    for img in images[split_idx:]:
        shutil.move(os.path.join(cls_path, img), os.path.join(val_cls_dir, img))

print("Images successfully moved to train/val folders.")


Images successfully moved to train/val folders.


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Paths
train_dir = '/content/RetinalOCT_Dataset/train'
val_dir = '/content/RetinalOCT_Dataset/val'

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)


In [ ]:
# Validation data just normalized
val_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 18400 images belonging to 9 classes.
Found 2800 images belonging to 9 classes.


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [ ]:
# Load pre-trained InceptionV3 without the top layer
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # freeze base


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:

# Build model on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(9, activation='softmax')  # 9 classes
])

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 284s 462ms/step - accuracy: 0.5087 - loss: 1.3848 - val_accuracy: 0.7755 - val_loss: 0.6765
Epoch 2/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 257s 447ms/step - accuracy: 0.7265 - loss: 0.7520 - val_accuracy: 0.7892 - val_loss: 0.5840
Epoch 3/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 261s 445ms/step - accuracy: 0.7714 - loss: 0.6321 - val_accuracy: 0.8193 - val_loss: 0.5170
Epoch 4/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 322s 560ms/step - accuracy: 0.7911 - loss: 0.5705 - val_accuracy: 0.8247 - val_loss: 0.4832
Epoch 5/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 324s 459ms/step - accuracy: 0.7950 - loss: 0.5600 - val_accuracy: 0.8244 - val_loss: 0.4859
Epoch 6/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 323s 461ms/step - accuracy: 0.8109 - loss: 0.5099 - val_accuracy: 0.8294 - val_loss: 0.4725
Epoch 7/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 261s 454ms/step - accuracy: 0.8196 - loss: 0.4907 - val_accuracy: 0.8229 - val_loss: 0.4668
Epoch 8/10
575/575 ━━━━━━━━━━━━━━━━━━━━ 263s 456ms/step - accuracy: 0.8204 -

In [ ]:
# Save the full model
model.save('/content/Retinal_OCT_C8_model.h5')
print("Model saved successfully!")


Model saved successfully!


In [ ]:
from google.colab import files
uploaded = files.upload()  # select retinal_cnn_model.h5


Saving Retinal_OCT_C8_model.h5 to Retinal_OCT_C8_model.h5


In [ ]:
import os

# Make sure 'models' folder exists
os.makedirs("models", exist_ok=True)


In [ ]:
model.save("models/Retinal_OCT_C8_model.keras")


In [ ]:
model.save("models/Retinal_OCT_C8_model.h5")


In [ ]:
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)       │ (None, 5, 5, 2048)     │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │         9,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,910,185 (91.21 MB)

 Trainable params: 2,107,401 (8.04 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
# Option 1: Save as H5
model.save("retinal_model.h5")




In [ ]:
from tensorflow import keras

# Load your old .h5 model
old_model = keras.models.load_model("retinal_model.h5", compile=False)

# ✅ Export as TensorFlow SavedModel
old_model.export("retinal_model_saved")

print("✅ Model converted to SavedModel format")


Saved artifact at 'retinal_model_saved'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 9), dtype=tf.float32, name=None)
Captures:
  133799502551504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133799901750928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133799901748048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133799901750160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133799901744400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133799901751696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133799901749584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133799901750352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133799901751312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133799901752464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1337999017

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil


# Zip the folder
shutil.make_archive("/content/retinal_model_saved", 'zip', "/content/retinal_model_saved")


'/content/retinal_model_saved.zip'